# Load Dataset with inital features

In [ ]:
import pandas as pd
import azureml.core
from azureml.core import Experiment, Dataset, Datastore, Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.compute import ComputeTarget, AmlCompute
import logging

ws = Workspace.from_config()

qualitydataset = Dataset.get_by_name(workspace=ws,name="iiot_quality_featured_data")
qualitydf = qualitydataset.to_pandas_dataframe()
print("Rows => {0}".format(qualitydf.shape[0]))
print("Columns => {0}".format(qualitydf.shape[1]))
qualitydf.head()

# Configure Experiment

In [ ]:
# Configure Cluster
cluster_name = "cpu-cluster"
compute_target = ComputeTarget(workspace=ws, name=cluster_name)

# Configure Automated ML settings
automl_settings = {
    "n_cross_validations": 3,
    "primary_metric": "average_precision_score_weighted",
    "enable_early_stopping": True,
    "max_concurrent_iterations": 2,  # This is a limit for testing purpose, please increase it as per cluster size
    "experiment_timeout_hours": 0.25,  # This is a time limit for testing purposes, remove it for real use cases, this will drastically limit ablity to find the best model possible
    "verbosity": logging.INFO,
}

label_column_name = "Quality"

automl_config = AutoMLConfig(
    task="classification",
    debug_log="automl_errors.log",
    compute_target=compute_target,
    training_data=qualitydataset,
    label_column_name=label_column_name,
    **automl_settings,
)

# Run Experiment

In [ ]:
experiment_name = "Baseline-Modeling-Automl"
experiment = Experiment(ws, experiment_name)

remote_run = experiment.submit(automl_config, show_output=False)